# How DCT and DST are Related to the DFT

The math can be confusing, and the [scipy docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.dst.html) can be especially confusing, so it's worth seeing **in code** *exactly* how the [DCT](https://en.wikipedia.org/wiki/Discrete_cosine_transform#Informal_overview) and the [DST](https://en.wikipedia.org/wiki/Discrete_sine_transform#Informal_overview) can be formed out of a DFT (FFT).

This is especially important if we start using [different variants](https://github.com/pavelkomarov/spectral-derivatives/blob/main/notebooks/dct_types_extension_experiments.ipynb) due to the DCT-II's apparent better [energy compaction](https://dsp.stackexchange.com/questions/96172/why-does-the-dct-ii-have-better-energy-compaction-than-dct-i).

In [1]:
import numpy as np
from scipy.fft import dct, idct, dst, idst, fft, ifft

First a vector to transform.

In [2]:
y_n = np.array([3, 7, -1, 4, 8])

## DCT-I

$$\begin{align}
[Y_0, ... Y_N] = \text{DCT-I}([y_0, y_1, ... y_N]) &= \text{FFT}([y_0, y_1, ... y_{N-1}, y_N, y_{N-1}, ... y_1])[:\!N+1] \\
[y_0, ... y_n] = \text{IDCT-I}([Y_0, Y_1, ... Y_N]) &= \frac{1}{M} \text{DCT-I}([Y_0, Y_1, ... Y_N]) = \text{IFFT}([Y_0, Y_1, ... Y_{N-1}, Y_N, Y_{N-1}, ... Y_{1}])[:\!N+1]
\end{align}
$$

The input to both the forward and inverse transforms can have the same structure, because the DCT-I is its own inverse up to factor $M = 2N$, which appears because the inverse transform is normalized by the length of the FFT input.

This case occurs both when transforming to frequency domain using DCT-I and when transforming back from frequency domain for even-order derivatives.

In [3]:
def dct_i_fft(y):
	y_ext = np.concatenate((y, y[-2:0:-1])) # even extension, ordering not sensitive like the odd case
	return fft(y_ext)[:len(y)]

def idct_i_ifft(Y):
	Y_ext = np.concatenate((Y, Y[-2:0:-1]))
	return ifft(Y_ext)[:len(Y)]

# Transform to frequency domain with the FFT and check this is actually the same as the DCT
Y_k = dct_i_fft(y_n)
assert np.all(np.abs(dct(y_n, 1) - Y_k) < 1e-10)

# Take a second derivative in frequency domain and check that the inverse transform of this is the same using DCT and FFT
Y_p2 = -Y_k*np.arange(len(Y_k))**2
assert np.all(np.abs(idct(Y_p2, 1) - idct_i_ifft(Y_p2)) < 1e-10)

# Ensure I have the correct normalization factor
M = 2*(len(Y_k) - 1)
assert np.all(np.abs(dct(Y_p2, 1)/M - idct_i_ifft(Y_p2)) < 1e-10)

## DST-I

$$\begin{align}
[Y_0, ... Y_N] = \text{DST-I}(j \cdot [y_0, y_1, ... y_N]) &= \text{FFT}([0, -y_0, -y_1, ... -y_{N-1}, -y_N, 0, y_N, y_{N-1}, ... y_1])[1\!:\!N] \\
[y_1, ... y_{N-1}] = \text{IDST-I}(j \cdot [Y_1, ... Y_{N-1}]) &= \frac{1}{M} \text{DST-I}(j \cdot [Y_1, ... Y_{N-1}]) = \text{IFFT}([0, Y_1, ... Y_{N-1}, 0, -Y_{N-1}, ... -Y_1])[1\!:\!N]
\end{align}
$$

Like for the DCT-I, $M = 2N$, and the DST-I is its own inverse up to this factor. However, unlike the DCT-I, `scipy`'s DST-I has funny indexing.

Notice the addition of $0$s at the $1^{st}$ and $\frac{M}{2}^{th}$ indices, which make the function odd: It doesn't make sense to sine transform a not-odd thing, and the DST-I assumes the function is odd around whole-number indices ahead of and behind the current signal, which means there must be zero crossings there. Presumably to save compute (but causing confusion in the process), `scipy`'s DST-I *does not take* the $0^{th}$ and $N^{th}$ indices.

Notice too that the negated locations of the input vector flip for the forward and inverse transforms. This is because in the forward FFT the beginning of the vector is multiplied by negative exponentials, and the back half is multiplied by positive exponentials, but in the inverse transform the front half is multiplied by positive exponentials, and the back half by negative exponentials. And we need a structure like $+x_k e^{+j} -x_k e^{-j}$ to form sines.

This case occurs when transforming odd-order derivatives back from frequency domain.

In [4]:
def dst_i_fft(y):
	y_ext = np.concatenate(([0], -y, [0], y[::-1])) # odd extension, with sign flip to account for fft rather than ifft
	return fft(y_ext)[1:1+len(y)]

def idst_i_ifft(Y):
	Y_ext = np.concatenate(([0], Y, [0], -Y[::-1]))
	return ifft(Y_ext)[1:1+len(Y)] # the input here is N-2 long, so to return 1...N-1 return len(Y) elements

# Verify the top identity holds for some arbitrary input
assert np.all(np.abs(dst(1j * y_n, 1) - dst_i_fft(y_n)) < 1e-10)

# Take a first derivative in frequency domain
Y_p1 = Y_k*1j*np.arange(len(Y_k))
Y_p1[-1] = 0 # zero out Nyquist term
# Y_p1 is now 0 at indices 0 and -1; the dst() function only takes the nonzero indices [1:-1] as input
assert np.all(np.abs(idst(1j*Y_p1[1:-1], 1) - idst_i_ifft(Y_p1[1:-1])) < 1e-10)

# Demonstrate M = 2N is correct
assert np.all(np.abs(dst(1j*Y_p1[1:-1], 1)/M - idst_i_ifft(Y_p1[1:-1])) < 1e-10)

## DCT-II/III

$$
\begin{align}
[Y_0, ... Y_N] = \text{DCT-II}([y_0, y_1, ... y_N]) &= \text{FFT}([y_0, y_1, ... y_N, y_N, ... y_0])[:\!N+1] \odot e^{-j\frac{2\pi}{M}\frac{1}{2}[0, ... N]}\\
&= \text{FFT}([0, y_0, 0, y_1, ... 0, y_N, 0, y_N, ... 0, y_0])[:\!N+1]\\
[y_0, ... y_n] = \text{IDCT-II}([Y_0, ... Y_N]) &= \frac{1}{M} \text{DCT-III}([Y_0, ... Y_N]) = \text{IFFT}([Y_0, Y_1, ... Y_N, 0, Y_N, Y_{N-1}, ... Y_1] \odot e^{j\frac{2\pi}{M}\frac{1}{2}[0, ... N, 0, -N, ... -1]})[:\!N+1]
\end{align}
$$

The DCT-II effectively adds a ["phase ramp"](https://dsp.stackexchange.com/questions/2807/fast-cosine-transform-via-fft). We stack palindrome of samples right next to each other in $n$, with a repeat.

The DCT-III doesn't use a Nyquist term (no $Y_{M/2} = Y_{N+1}$), so the IFFT gets a 0 there. We undo the phase ramp before transforming back.

This case is important for obtaining the frequency representation and for recovering even-order derivatives if we use the DCT-II as the forward transform.

In [5]:
def dct_ii_fft(y):
	y_ext = np.concatenate((y, y[::-1]))
	return fft(y_ext)[:len(y)] * np.exp(-1j*np.pi*np.arange(len(y))/(2*len(y)))

def dct_ii_fft_spaced(y):
    y_ext = np.concatenate((y, y[::-1]))
    y_spaced = [0 if i%2==0 else y_ext[(i-1)//2] for i in range(4*len(y))]
    return fft(y_spaced)[0:len(y)]

def dct_iii_ifft(Y):
	M = 2*len(Y)
	Y_pos = Y * np.exp(1j * np.pi/M * np.arange(len(Y)))
	Y_neg = Y * np.exp(-1j * np.pi/M * np.arange(len(Y)))
	Y_ext = np.concatenate((Y_pos, [0], Y_neg[-1:0:-1]))
	return ifft(Y_ext)[:len(Y)]

# Transform to frequency domain with the FFT and check this is actually the same as the DCT
Y_k = dct_ii_fft(y_n)
assert np.all(np.abs(dct(y_n, 2) - Y_k) < 1e-10)
assert np.all(np.abs(dct_ii_fft_spaced(y_n) - Y_k) < 1e-10)

# Take a second derivative in frequency domain and check that the inverse transform of this is the same using DCT and FFT
Y_p2 = -Y_k*np.arange(len(Y_k))**2
assert np.all(np.abs(dct_iii_ifft(Y_p2) - idct(Y_p2, 2)) < 1e-10)

# Demonstrate this is the correct M
M = 2*len(y_n)
assert np.all(np.abs(idct(Y_k, 2) - dct(Y_k, 3)/M) < 1e-10)

## DST-II/III

$$\begin{align}
[Y_0, ... Y_N] = \text{DST-II}(j \cdot [y_0, y_1, ... y_N]) &= \text{FFT}([-y_0, -y_1, ... -y_N, y_N, ... y_0])[1\!:\!N+2] \odot e^{-j\frac{2\pi}{M}\frac{1}{2}[1, ... N+1]} \\
[y_0, ... y_N] = \text{IDST-II}(j \cdot [Y_1, ... Y_N]) &= \frac{1}{M} \text{DST-III}(j \cdot [Y_1, ... Y_N]) = \text{IFFT}([0, Y_1, ... Y_{N-1}, Y_{N}, -Y_{N-1}, ... -Y_1] \odot e^{j\frac{2\pi}{M}\frac{1}{2}[0, ... N-1, N, -(N-1), ... -1]})[:\!N+1]
\end{align}
$$

This case is important for taking odd-order derivatives with the DCT-II as the forward transform.

In [6]:
def dst_ii_fft(y):
    y_ext = np.concatenate((-y, y[::-1]))
    return fft(y_ext)[1:len(y)+1] * np.exp(-1j*np.pi*np.arange(1,1+len(y))/(2*len(y)))

def dst_ii_fft_spaced(y):
	y_ext = np.concatenate((-y, y[::-1])) # odd extension, with sign flip to account for fft rather than ifft
	y_spaced = [0 if i%2==0 else y_ext[(i-1)//2] for i in range(4*len(y))]
	return fft(y_spaced)[1:1+len(y)]

def dst_iii_ifft(Y): # giving it only k \in {1,...N}
    M = 2*len(Y) # M = 2N for the DST-III
    Y_pos = Y * np.exp(1j * np.pi/M * np.arange(1,1+len(Y))) # k in {1,...N} here
    Y_neg = Y[:-1] * -np.exp(-1j * np.pi/M * np.arange(1,len(Y))) # k in {1,...N-1} here
    Y_ext = np.concatenate(([0], Y_pos, Y_neg[::-1]))
    return ifft(Y_ext)[:len(Y)] # we return from the 0th index, because answer corresponds to all input

# Verify the top identity holds for some arbitrary input
assert np.all(np.abs(dst(1j*y_n, 2) - dst_ii_fft(y_n)) < 1e-10)
assert np.all(np.abs(dst(1j*y_n, 2) - dst_ii_fft_spaced(y_n)) < 1e-10)

# Take a first derivative in frequency domain
Y_p1 = Y_k*1j*np.arange(len(Y_k))
M = 2*len(Y_p1[1:]) # 0, Y_1, ... Y_N stacks on itself, -Y_{N-1}, ... -Y_1 = M things = 2N, not 2(N+1) like in the DCT case!
assert np.all(np.abs(idst(1j * Y_p1[1:], 2) - dst(1j * Y_p1[1:], 3)/M) < 1e-10)

# Y_p1 is now 0 at index 0; the dst() function only takes the nonzero indices [1:] as input
assert np.all(np.abs(dst(1j*Y_p1[1:], 3)/M - dst_iii_ifft(Y_p1[1:])) < 1e-10)